# reading the query results from SQL

In [ ]:
# reading the query result
filename='demographics/orca_note.txt'
lines=[line.strip() for line in open(filename).read().split('\n') if line.strip()]
keys=[key.strip() for key in lines[0].split('\t') if key]

output={}
for line in lines[1:]:
    values=[v.strip() for v in line.split('\t')]
    assert len(values)==len(keys)
    output[values[1]]={}
    for k,v in zip(keys,values):
        output[values[1]][k]=v

# for t in patient_ids:
#     assert patient_ids.count(t)==1,patient_ids.count(t)
print(len(output))

In [ ]:
query=f'use r21_symptom;\nselect * from demographic where study_id in ({",".join(set(output.keys()))});'
with open('demographics/demographics_query.txt','w') as f:
    f.write(query)

In [ ]:
# summarizing the output
filename='demographics/demographics.txt'
lines=[line.strip() for line in open(filename).read().split('\n') if line.strip()]
keys=[key.strip().lower() for key in lines[0].split('\t') if key]

for line in lines[1:]:
    values=[v.strip() for v in line.split('\t')]
    assert len(values)==len(keys)
    id=values[0]
    for dk in output:
        if output[dk][keys[0]]==id:
            for k,v in zip(keys,values):
                output[dk][k]=v

# for t in patient_ids:
#     assert patient_ids.count(t)==1,patient_ids.count(t)
import json
print(len(output))
with open('demographics/summary.json','w') as f:
    json.dump(output,f,indent=4)

# split the dataset into train/valid/test

In [87]:
# spliting 
import json
from collections import Counter
import random
import shutil
import os


for split in ['train','valid','test']:
    #shutil.rmtree(f'R15_dataset_paper/final4/{split}')
    os.makedirs(f'R15_dataset_paper/final4/{split}',exist_ok=True)


demos=json.loads(open('demographics/summary.json').read())
split='test'
output={
    'test':115,
    'valid':47
}

# split='train'
# output={
#     'train':400,
#     'valid':13
# }

ids=glob(f'R15_dataset_paper/final3/{split}_cleaned/*.ann')
doc_ids=[id.split('/')[-1].split('.')[0] for id in ids]
ppl_ids=[demos[id]["study_id"] for id in doc_ids]

if split=='test':
    total=0
    valid_pids=[]
    for sid,count in Counter(ppl_ids).most_common():
        if count>=2:
            total+=count
            valid_pids.append(sid)
else:
    valid_pids=['3550','2203','3741'] 
    total=8
valid_ids=[id for idx,id in enumerate(doc_ids) if ppl_ids[idx] in valid_pids]


single_idx=[doc_ids[idx] for idx,pid in enumerate(ppl_ids) if ppl_ids.count(pid)==1 and pid not in valid_pids]
valid_ids.extend(single_idx[:output['valid']-total])


for id in doc_ids:
    if id not in valid_ids:
        shutil.copy(f'R15_dataset_paper/final3/{split}_cleaned/{id}.ann',f'R15_dataset_paper/final4/{split}/{id}.ann')
        shutil.copy(f'R15_dataset_paper/final3/{split}_cleaned/{id}.txt',f'R15_dataset_paper/final4/{split}/{id}.txt') 
    else:
        shutil.copy(f'R15_dataset_paper/final3/{split}_cleaned/{id}.ann',f'R15_dataset_paper/final4/valid/{id}.ann')
        shutil.copy(f'R15_dataset_paper/final3/{split}_cleaned/{id}.txt',f'R15_dataset_paper/final4/valid/{id}.txt')      

In [91]:
# checking the numbers 
from glob import glob
import os
import shutil
train_ids=[file.split('/')[-1].split('.')[0] for file in \
           glob('R15_dataset_paper/final4/train/*.txt')]
assert len(train_ids)==len(set(train_ids))
valid_ids=[file.split('/')[-1].split('.')[0] for file in \
           glob('R15_dataset_paper/final4/valid/*.txt')]
assert len(train_ids)==len(set(train_ids))
test_ids=[file.split('/')[-1].split('.')[0] for file in \
           glob('R15_dataset_paper/final4/test/*.txt')]
assert len(train_ids)==len(set(train_ids))
print(len(train_ids),len(valid_ids),len(test_ids))

# checking for non-overlap
train_pids=[demos[id]["study_id"] for id in train_ids]
valid_pids=[demos[id]["study_id"] for id in valid_ids]
test_pids=[demos[id]["study_id"] for id in test_ids]
print(len(set(train_pids)),len(set(valid_pids)),len(set(test_pids)))

assert [id for id in train_pids if id in valid_pids]==[], [id for id in train_pids if id in valid_pids]
assert [id for id in train_pids if id in test_pids]==[]

assert [id for id in valid_pids if id in train_pids]==[]
assert [id for id in valid_pids if id in test_pids]==[]

assert [id for id in test_pids if id in train_pids]==[]
assert [id for id in test_pids if id in valid_pids]==[]


400 60 115
306 43 115


# outputing the summaries

In [92]:
import json
demographics=json.loads(open('demographics/summary.json').read())

all_ids={
    'train':train_ids,
    'valid':valid_ids,
    'test':test_ids
}
# note types
with open('demographics/summary.csv','w') as f:
    f.write('Type,Subtype,Train,Valid,Test\n')
    for key in ['event_cd_descr','prov_specialty','sex','dlbclpatient','prostatecancerpatient']:
        types=set([dic[key] for v,dic in demographics.items()])
        counts=[]
        for type in types:
            f.write(f'{key},{type},')
            for split in all_ids:
                subset=[dic[key] for v,dic in demographics.items() if v in all_ids[split]]
                f.write(f'{subset.count(type)},')
            f.write('\n')
        unique_patients=[]
        for split in all_ids:
            unique_patients.append([dic["study_id"] for v,dic in demographics.items() if v in all_ids[split]])
        for i,j in [[0,2],[1,2],[2,1],[2,0]]:
                for t1 in unique_patients[i]:
                    assert t1 not in unique_patients[j], [i,j]
        f.write(f'#unique patients,-,{len(set(unique_patients[0]))},{len(set(unique_patients[1]))},{len(set(unique_patients[2]))}')
        f.write('\n')

# summarizing the counts

In [2]:
from glob import glob
exps=['train','valid','test']
for exp in exps:
        scores={
        "Drug,Trigger":[0,0,0],
        "Problem,Trigger":[0,0,0],
        "Problem,Assertion":[0,0,0],
        "Problem,Change":[0,0,0],
        "Problem,Severity":[0,0,0],
        "Problem,Anatomy":[0,0,0],
        "Problem,Characteristics":[0,0,0],
        "Problem,Duration":[0,0,0],
        "Problem,Frequency":[0,0,0],
        }

        for file in glob('../dataset/BRAT/*.csv'):
            if exp in file and '_detailed.csv' in file:
                lines=open(file).read().split('\n')[1:]
                out_ids=[]
                for line in lines:
                    if 'Problem,Trigger' in line and float(line.split(',')[-1])==0.5:
                        out_ids.append(line.split(',')[0])
                    
                for line in lines:
                    if line:
                        words=line.split(',')[1:]
                        key=','.join(words[:2])
                        if key not in scores:
                            #print(key)
                            continue
                        if any([id in line for id in out_ids]):
                            #print(line)
                            continue
                        if words[3]!='0':
                            for i in range(3):
                                scores[key][i]+=int(words[i+3])
        scores['Overall,-']=[0,0,0]
        for i in range(3):
            scores['Overall,-'][i]=sum([v[i] for k,v in scores.items()])

        with open(f'../performance/entity_counts_{exp}.csv','w') as f:
            f.write('event,Argument,NT,NP,TP,P,R,F1\n')
            for key in scores:     
                f.write(key+','+','.join([str(s) for s in scores[key]]))
                P=scores[key][2]*100.0/scores[key][1] if scores[key][1] else 0
                R=scores[key][2]*100.0/scores[key][0] if scores[key][0] else 0
                F1=2*P*R/(P+R) if R+P else 0
                f.write(f",{round(P,1)},{round(R,1)},{round(F1,1)}\n")